In [1]:
from sudoku_mrv import generate_board, verify_board

In [6]:
a = generate_board(completeness=20, outer_grid_size=9)

In [7]:
a

[[6, 4, 3, 7, 2, 9, 8, 1, 5],
 [9, 2, 5, 0, 0, 0, 4, 0, 0],
 [1, 7, 8, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [4]:
verify_board(a)

True

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from PIL import Image
import io
from matplotlib import gridspec


def visualize_sudoku(board, title=None, cmap=None, show_values=True, figsize=(6, 6), outer_grid_size=9):
    """
    Visualize a Sudoku board with different colors for each number.
    
    Args:
        board: A 9x9 numpy array or list of lists representing the Sudoku board
        title: Optional title for the plot
        cmap: Optional custom colormap (default is a pastel colormap)
        show_values: Whether to display the numerical values in cells
        figsize: Size of the figure (width, height) in inches
        
    Returns:
        A PIL Image of the visualization
    """
    # Create a new figure for each board to prevent any sharing
    plt.clf()  # Clear the current figure
    plt.close('all')  # Close all figures
    fig = plt.figure(figsize=(6, 6))

    # Convert to numpy array if it's a list
    if isinstance(board, list):
        board = np.array(board)
    
    # Create a default colormap if none provided
    if outer_grid_size == 9:
        if cmap is None:
            # Create a colormap with 10 colors (0-9, where 0 is empty)
            colors = [
                '#FFFFFF',  # 0: White (empty)
                '#FFB3BA',  # 1: Light pink
                '#FFDFBA',  # 2: Light orange
                '#FFFFBA',  # 3: Light yellow
                '#BAFFC9',  # 4: Light green
                '#BAE1FF',  # 5: Light blue
                '#D0BAFF',  # 6: Light purple
                '#FFB3F6',  # 7: Light magenta
                '#C4C4C4',  # 8: Light gray
                '#FFD700',  # 9: Gold - changed from light cyan
            ]  
            cmap = ListedColormap(colors)
    else:
        # Create a colormap with colors from red to blue for numbers 0 to outer_grid_size
        # White for 0 (empty cells)
        colors = ['#FFFFFF']  
        # Linear interpolation from red to blue for numbers 1 to outer_grid_size
        for i in range(outer_grid_size):
            r = int(255 * (outer_grid_size - i) / outer_grid_size)
            b = int(255 * i / outer_grid_size)
            colors.append(f'#{r:02x}00{b:02x}')
        cmap = ListedColormap(colors)
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot the board
    im = ax.imshow(board, cmap=cmap, vmin=0, vmax=outer_grid_size)

    inner_grid_size = int(outer_grid_size ** 0.5)
    
    # Add grid lines
    for i in range(outer_grid_size+1):
        lw = 2 if i % inner_grid_size == 0 else 0.5
        ax.axhline(i - 0.5, color='black', linewidth=lw)
        ax.axvline(i - 0.5, color='black', linewidth=lw)
    
    # Add values to cells if requested
    if show_values:
        for i in range(outer_grid_size):
            for j in range(outer_grid_size):
                if board[i, j] != 0:
                    ax.text(j, i, str(board[i, j]), ha='center', va='center', 
                            fontsize=12, fontweight='bold')
    
    # Remove ticks
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add title if provided
    if title:
        ax.set_title(title)
    
    plt.tight_layout()
    
    # Convert to PIL image
    pil_image = fig_to_pil(fig)
    plt.close(fig)  # Close the figure to avoid displaying it
    return pil_image


def fig_to_pil(fig):
    """Convert a matplotlib figure to a PIL Image"""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img = Image.open(buf)
    return img

In [10]:
# trainer

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import random

# Custom dataset for generating Sudoku boards
class SudokuDataset(Dataset):
    def __init__(self, num_samples=10000, board_size=9):
        self.num_samples = num_samples
        self.board_size = board_size
        
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # For now, we'll use a simple approach to generate valid Sudoku boards
        # In a real implementation, you might want to use a more sophisticated generator
        # completeness = int(torch.rand(1).item() * 100)
        board = generate_board(completeness=100, outer_grid_size=self.board_size)
        board = torch.tensor(board)
        return board

# Training function
def train_diffusion_model(
    model, 
    outer_grid_size=9, 
    num_epochs=10, 
    batch_size=32, 
    lr=1e-4, 
    device="cuda" if torch.cuda.is_available() else "mps", 
    eval_every_n_step=100, 
    warmup_steps = 200, 
    compiled=False
):
    # Create dataset and dataloader
    dataset = SudokuDataset(board_size=outer_grid_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    
    # Setup optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.997), weight_decay=0.01)
    
    # warmp up lr scheduler
    # Calculate total steps for the entire training
    total_steps = len(dataloader) * num_epochs
    
    # Create a learning rate scheduler with linear warmup and linear decay
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            # Linear warmup phase
            return float(current_step) / float(max(1, warmup_steps))
        else:
            # Linear decay phase
            return max(0.0, float(total_steps - current_step) / float(max(1, total_steps - warmup_steps)))
    
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    
    # Move model to device
    model = model.to(device)
    if compiled:
        model = torch.compile(model)

    total_step = 0
    
    # Training loop
    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0
        
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}")
        for batch_idx, boards in enumerate(progress_bar):
            model.train()
            boards = boards.to(device)
            optimizer.zero_grad(set_to_none=True)
            preds_bld, loss = model.forward_loss(boards)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item(), "lr": optimizer.param_groups[0]['lr']})

            total_step += 1
            if total_step % eval_every_n_step == 0:
                model.eval()
                # sample
                with torch.no_grad():
                    generated_boards = model.generate(batch_size=10)
                    boards = generated_boards.chunk(generated_boards.shape[0], dim=0)
                    boards = [b.squeeze(0).tolist() for b in boards]
                    results = []
                    board_figs = []
                    for board in boards:
                        results.append(verify_board(board, outer_grid_size=model.outer_grid_size))
                        board_figs.append(visualize_sudoku(board, outer_grid_size=model.outer_grid_size))
                    print(results)
                    canvas_width = board_figs[0].width * len(boards)
                    canvas_height = board_figs[0].height

                    # create canvas
                    canvas = Image.new("RGB", (canvas_width, canvas_height), 'white')
                    for i, board_fig in enumerate(board_figs):
                        canvas.paste(board_fig, (board_fig.width * i, 0))
                    canvas.save(f"generated_boards_{total_step}.png")
                    canvas.close()
                    
                    # Clean up
                    for fig in board_figs:
                        fig.close()
    
    return model

# Example usage
outer_grid_size = 9
model = DiscreteDiffusion(outer_grid_size=outer_grid_size)
trained_model = train_diffusion_model(model, outer_grid_size=outer_grid_size)

100%|███████████████████████████████████████████████████████████████████████████████████| 128/128 [00:05<00:00, 21.78it/s]


[False, False, False, False, False, False, False, False, False, False]


100%|███████████████████████████████████████████████████████████████████████████████████| 128/128 [00:02<00:00, 45.45it/s]


[False, False, False, False, False, False, False, False, False, False]


100%|███████████████████████████████████████████████████████████████████████████████████| 128/128 [00:02<00:00, 45.49it/s]


...

[True, True, True, True, True, True, True, True, True, True]


100%|███████████████████████████████████████████████████████████████████████████████████| 128/128 [00:02<00:00, 44.66it/s]


[True, True, True, True, True, True, True, True, True, True]


Epoch 10/10: 100%|█████████████████████████████████████████████████████| 313/313 [02:26<00:00,  2.14it/s, loss=1.87, lr=0]


<Figure size 600x600 with 0 Axes>